In [149]:
import os
import numpy as np
import pandas as pd
from PIL import Image

CSV_PATH='data/etl4/ETL4C.csv'
IMAGE_PATH='data/etl4/ETL4C_00.png'

HIRAGANA = np.array(list(set('あいうえおかきくけこさしすせそたちつてとなにぬねのはひふへほまみむめもやいゆえよらりるれろわゐうゑをん')))
HIRAGANA_NB=51
WRITERS_NB=120
TOTAL_NB=HIRAGANA_NB*WRITERS_NB

MALE=1
FEMALE=2

In [150]:
df = pd.read_csv(CSV_PATH)
df.head()

,Serial Data Number,Serial Sheet Number,JIS Code,EBCDIC Code,4 Character Code,Evaluation of Individual Character Image,Evaluation of Character Group,Sample Position Y on Sheet,Sample Position X on Sheet,Male-Female Code,Age of Writer,Industry Classification Code,Occupation Classification Code,Sheet Gatherring Date,Scanning Date,Number of X-Axis Sampling Points,Number of Y-Axis Sampling Points,Number of Levels of Pixel,Magnification of Scanning Lenz,Serial Data Number (old)
0,500100,5001,b1,81,H A,0,0,1,0,1,23,9144,11,741202,741216,72,76,16,133,0
1,500101,5001,b2,82,H I,0,0,1,1,1,23,9144,11,741202,741216,72,76,16,133,0
2,500102,5001,b3,83,H U,0,0,1,2,1,23,9144,11,741202,741216,72,76,16,133,0
3,500103,5001,b4,84,H E,0,0,1,3,1,23,9144,11,741202,741216,72,76,16,133,0
4,500104,5001,b5,85,H O,0,0,1,4,1,23,9144,11,741202,741216,72,76,16,133,0


In [151]:
# Find a way to identify identical characters using a common key
id_key = None

for key in df.keys():
    keys_occ = df.groupby(by=key)[key].count().values
    differents_keys_length = keys_occ.size
    if np.all(keys_occ == WRITERS_NB) and differents_keys_length == HIRAGANA_NB:
        id_key = key
        
id_key

'4 Character Code'

In [152]:
# Clean dataset
X_PIXELS_LENGTH = df['Number of X-Axis Sampling Points'].values[0]
Y_PIXELS_LENGTH = df['Number of Y-Axis Sampling Points'].values[0]
PIXEL_LEVEL = df['Number of Y-Axis Sampling Points'].values[0]

del df['Serial Sheet Number']
del df['JIS Code']
del df['EBCDIC Code']
del df['Evaluation of Individual Character Image']
del df['Evaluation of Character Group']
del df['Industry Classification Code']
del df['Occupation Classification Code']
del df['Sheet Gatherring Date']
del df['Scanning Date']
del df['Magnification of Scanning Lenz']
del df['Serial Data Number (old)']
del df['Number of X-Axis Sampling Points']
del df['Number of Y-Axis Sampling Points']
del df['Number of Levels of Pixel']

df = df.rename(columns={
    'Serial Data Number': 'serial',
    '4 Character Code': 'character_code',
    'Sample Position Y on Sheet': 'x',
    'Sample Position X on Sheet': 'y',
    'Male-Female Code': 'gender',
    'Age of Writer': 'writer_age',
})

df.head()

,serial,character_code,x,y,gender,writer_age
0,500100,H A,1,0,1,23
1,500101,H I,1,1,1,23
2,500102,H U,1,2,1,23
3,500103,H E,1,3,1,23
4,500104,H O,1,4,1,23


In [192]:
# Set hiragana caracters for each rows in dataframe
get_hiragana = lambda x: print(x.name % HIRAGANA.size)
df.apply(get_hiragana, axis=1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
0
1
2
3
4
5
6
7
8
9


0       None
1       None
2       None
3       None
4       None
        ... 
6115    None
6116    None
6117    None
6118    None
6119    None
Length: 6120, dtype: object

In [191]:
t = [4, 5]
a = [1, 2, 3, 4]
for x in a:
    i = a.index(x)
    print(t[i % len(t)])
    t[0]

4
5
4
5


In [203]:
list(set('あいうえおかがきぎくぐけげこごさざしじすずせぜそぞただちぢつづてでとどなにぬねのはばぱひびぴふぶぷへべぺほぼぽまみむめもやゆよらりるれろわをん'))

['せ',
 'び',
 'て',
 'げ',
 'ぺ',
 'そ',
 'の',
 'よ',
 'ろ',
 'す',
 'み',
 'ぴ',
 'ゆ',
 'お',
 'め',
 'さ',
 'た',
 'ぢ',
 'ぼ',
 'ご',
 'う',
 'ま',
 'が',
 'き',
 'く',
 'ぞ',
 'え',
 'づ',
 'ぱ',
 'ぶ',
 'ど',
 'け',
 'ち',
 'な',
 'ぷ',
 'ぽ',
 'れ',
 'や',
 'は',
 'し',
 'こ',
 'ば',
 'に',
 'べ',
 'ね',
 'で',
 'い',
 'ひ',
 'り',
 'ざ',
 'ぬ',
 'を',
 'ら',
 'る',
 'ぜ',
 'ふ',
 'ず',
 'ほ',
 'へ',
 'か',
 'だ',
 'つ',
 'も',
 'ん',
 'ぎ',
 'と',
 'ぐ',
 'わ',
 'あ',
 'じ',
 'む']

In [204]:
len(label)

71